<a href="https://colab.research.google.com/github/sarmadsaleem333/Simple-Continual-Learning-Pipeline-/blob/main/CL_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Continual Learning Pipeline implementation on MNIST Dataset

**Continual learning** (also known as **lifelong learning**) refers to the ability of a machine learning model or system to continuously learn and adapt to new information or tasks over time, without forgetting previously learned knowledge. Unlike traditional machine learning, which typically trains on a fixed dataset and then remains static, continual learning allows models to evolve and improve as they are exposed to new data.

Key characteristics of continual learning include:

1. **Learning from New Data**: The model is capable of incorporating new information or tasks while maintaining performance on earlier tasks.
  
2. **Avoiding Catastrophic Forgetting**: A major challenge in continual learning is ensuring that the model doesn't forget previously learned tasks when learning new ones.

3. **Adaptation to Changing Environments**: The model can adjust to changes in data distribution or new tasks that weren't initially part of its training.

4. **Efficiency**: The system should be able to learn and update efficiently, often with limited computational resources and without requiring retraining from scratch.

In practice, continual learning is important in areas such as robotics, autonomous systems, and real-time data analysis, where new experiences or information continually emerge.

# Implementation using Avalanche

**Avalanche** is a framework designed to support **continual learning** in machine learning. It provides tools and libraries for building, testing, and evaluating continual learning models, helping researchers and practitioners develop algorithms that can learn over time without forgetting previously acquired knowledge.

### Key Features of Avalanche:
1. **Task-Incremental Learning**: It supports various types of continual learning paradigms, such as task-incremental learning, where the model encounters new tasks over time and needs to retain knowledge from previous tasks.
  
2. **Support for Different Scenarios**: Avalanche provides flexibility in handling different scenarios in continual learning, including:
   - **Class-incremental learning**: Where classes are introduced one by one.
   - **Domain-incremental learning**: Where the domain or distribution changes over time, such as in changing environments or evolving data distributions.

3. **Benchmarking**: The framework offers tools to evaluate models on several well-established continual learning benchmarks, allowing researchers to test the effectiveness of their models and compare them against others in the field.

4. **Memory Management**: Since continual learning often involves a challenge of avoiding **catastrophic forgetting**, Avalanche provides tools to implement memory mechanisms, such as replay buffers, to help retain useful information from past tasks.

5. **Flexible Integration**: Avalanche is compatible with popular deep learning libraries like PyTorch, making it easy to integrate into existing workflows.

6. **Metrics and Evaluation**: The framework offers a range of evaluation metrics, such as accuracy, forgetting, and others that are specifically useful in continual learning scenarios.

### Why Avalanche is Important:
Continual learning poses several challenges, notably **catastrophic forgetting**, where models forget previously learned tasks or knowledge when exposed to new tasks. Avalanche aids in developing and testing solutions to this problem by providing a structured environment to experiment with strategies such as:
- **Replay methods**, where previously encountered data or tasks are replayed to avoid forgetting.
- **Regularization techniques**, which help preserve older knowledge while allowing the model to adapt to new tasks.

By using Avalanche, researchers can advance our understanding of how models can learn continuously and effectively over time, making it a valuable tool in the field of **lifelong learning**.

# Installation

In [ ]:
! pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-z_8zj4x3
  Running command git clone --filter=blob:none --quiet https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-z_8zj4x3
  Resolved https://github.com/ContinualAI/avalanche.git to commit 625e46d9203878ed51457f6d7cd8d4e9fb05d093
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.2/134.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.2/585.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.6/558.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

# Implementation

Importing Libraries

In [ ]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive


Creating Benchmark

In [ ]:
# here it is creating benchmark using 5 tasks as [0,1],[2,3],[4,5],[6,7],[8,9]

scenario = SplitMNIST(n_experiences=5)


Create The Model

In [ ]:
model = SimpleMLP(num_classes=scenario.n_classes)


Define Evaluation Plugin and Loggers

In [ ]:
# Loggers to record and visualize metrics
tb_logger = TensorboardLogger()
# for storing logs in text file
text_logger = TextLogger(open('log.txt', 'a'))
# printing Logs
interactive_logger = InteractiveLogger()

# Evaluation plugin to manage and log metrics
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)


# Define the Naive Strategy
**Naive**: This is a basic strategy for training a model on each task. In continual learning, the Naive strategy doesn't use any techniques like replay or regularization to prevent forgetting. It just trains the model on the current task.

**SGD**: Stochastic Gradient Descent optimizer with a learning rate of 0.001 and momentum of 0.9.

CrossEntropyLoss: Loss function used for classification tasks.

train_mb_size=500: The mini-batch size during training (500 samples).

train_epochs=1: The number of epochs (1 epoch per experience).

eval_mb_size=100: The mini-batch size during evaluation.

evaluator=eval_plugin: The evaluation plugin to track and log metrics during training.

In [ ]:
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)


/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


# Training Loop

In [ ]:
# Start training loop
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # Train on the current experience
    res = cl_strategy.train(experience)
    print('Training completed')

    # Evaluate on the entire test set
    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(scenario.test_stream))


Starting experiment...
Start of experience:  0
Current Classes:  [4, 5]
-- >> Start of training phase << --
100%|██████████| 23/23 [00:03<00:00,  6.72it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56512.3223
	DiskUsage_MB/train_phase/train_stream/Task000 = 56512.3223
	Loss_Epoch/train_phase/train_stream/Task000 = 1.3529
	Loss_MB/train_phase/train_stream/Task000 = 0.4106
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0120
	Time_Epoch/train_phase/train_stream/Task000 = 3.4232
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6183
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9430
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 26.95it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 88.5307
	DiskUsage_Exp/eval


# Explanation
As we have seen above we used simple Naive approach and the model accuracy decreases on previoulsy learned classes so we are going to use some CL approaches to stop Forgetting

# Regularization EWC
# 3.1. Elastic Weight Consolidation (EWC)
EWC prevents forgetting by adding a penalty term to the loss function that penalizes large changes to important weights (those that are critical for previous tasks).

**Pros: **Prevents large weight changes and stabilizes learning on new tasks.

**Cons:** Requires calculating the importance of weights, which can be computationally expensive.

In [ ]:
from avalanche.training.plugins import EWCPlugin

# Initialize the EWC plugin with a regularization term strength (lambda)
ewc = EWCPlugin(ewc_lambda=0.1)

# Initialize the Naive strategy with the EWC plugin
cl_strategy_ewc = Naive(
    model,
    SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(),
    train_mb_size=500,
    train_epochs=1,
    eval_mb_size=100,
    evaluator=eval_plugin,  # Only pass the evaluator once
    plugins=[ewc]  # Add the EWC plugin here
)


/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


In [ ]:

# Start training loop
print('Starting experiment with EWC...')

results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # Train on the current experience using EWC regularization
    res = cl_strategy_ewc.train(experience)
    print('Training completed')

    # Evaluate on the entire test set
    print('Computing accuracy on the whole test set')
    results.append(cl_strategy_ewc.eval(scenario.test_stream))


Starting experiment with EWC...
Start of experience:  0
Current Classes:  [4, 5]
-- >> Start of training phase << --
100%|██████████| 23/23 [00:02<00:00,  8.35it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56613.2100
	DiskUsage_MB/train_phase/train_stream/Task000 = 56613.2100
	Loss_Epoch/train_phase/train_stream/Task000 = 2.1345
	Loss_MB/train_phase/train_stream/Task000 = 1.2248
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0079
	Time_Epoch/train_phase/train_stream/Task000 = 2.7506
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2470
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8555
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 32.05it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 89.2450
	DiskUsage

# Experience Replay

The model stores a small subset of previous data and replays it alongside the new data while training on each new task.


**Pros**: Helps in maintaining knowledge about previous tasks.


**Cons**: Requires storage for past examples, and replaying large amounts of data may increase computation.

In [41]:
from avalanche.training.plugins import ReplayPlugin

replay = ReplayPlugin(mem_size=100)  # Store 100 examples from previous tasks


cl_strategy_replay= Naive(
    model,
    SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(),
    train_mb_size=500,
    train_epochs=1,
    eval_mb_size=100,
    evaluator=eval_plugin,
    plugins=[replay]
)



/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


In [42]:

# Start training loop
print('Starting experiment with Replay...')

results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # Train on the current experience using EWC regularization
    res = cl_strategy_replay.train(experience)
    print('Training completed')

    # Evaluate on the entire test set
    print('Computing accuracy on the whole test set')
    results.append(cl_strategy_replay.eval(scenario.test_stream))


Starting experiment with Replay...
Start of experience:  0
Current Classes:  [4, 5]
-- >> Start of training phase << --
100%|██████████| 23/23 [00:02<00:00,  7.89it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56709.1484
	DiskUsage_MB/train_phase/train_stream/Task000 = 56709.1484
	Loss_Epoch/train_phase/train_stream/Task000 = 1.6060
	Loss_MB/train_phase/train_stream/Task000 = 0.6359
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0157
	Time_Epoch/train_phase/train_stream/Task000 = 2.9160
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4147
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9125


/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 24.45it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 84.3914
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56709.1484
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.5135
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9328
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 22.05it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 83.6748
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 56713.1484
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.2534
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 1.0918
	Top1_Acc_Exp/eval_phase/test_stream/Task0

/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)



Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56725.1484
	DiskUsage_MB/train_phase/train_stream/Task000 = 56725.1484
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7016
	Loss_MB/train_phase/train_stream/Task000 = 0.4035
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0081
	Time_Epoch/train_phase/train_stream/Task000 = 3.5782
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8208
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9116
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 29.93it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 89.2080
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56737.1143
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.0240
	Loss_Exp/eval_phase/test_stream/Task00

...


Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56753.1143
	DiskUsage_MB/train_phase/train_stream/Task000 = 56753.1143
	Loss_Epoch/train_phase/train_stream/Task000 = 1.2898
	Loss_MB/train_phase/train_stream/Task000 = 0.5749
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0092
	Time_Epoch/train_phase/train_stream/Task000 = 4.1923
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5565
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8611
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 20.12it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 83.1801
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56753.1143
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.3111
	Loss_Exp/eval_phase/test_stream/Task00

/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)



Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56769.1143
	DiskUsage_MB/train_phase/train_stream/Task000 = 56769.1143
	Loss_Epoch/train_phase/train_stream/Task000 = 0.9445
	Loss_MB/train_phase/train_stream/Task000 = 0.5153
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0078
	Time_Epoch/train_phase/train_stream/Task000 = 3.7763
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6869
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.8488
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 29.76it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 90.3046
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56769.1143
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.7689
	Loss_Exp/eval_phase/test_stream/Task00

/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)



Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56793.0742
	DiskUsage_MB/train_phase/train_stream/Task000 = 56793.0742
	Loss_Epoch/train_phase/train_stream/Task000 = 1.4403
	Loss_MB/train_phase/train_stream/Task000 = 0.7882
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0295
	Time_Epoch/train_phase/train_stream/Task000 = 6.3106
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4865
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.7460
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 30.66it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 91.4924
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56793.0742
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.7257
	Loss_Exp/eval_phase/test_stream/Task00

# Hybrid Technique

# 4.1. Replay + EWC
This combines experience replay with EWC to replay past experiences and regularize the model's weights to preserve knowledge of old task

In [46]:
replay = ReplayPlugin(mem_size=100)
ewc = EWCPlugin(ewc_lambda=0.001)


cl_strategy_hybrid= Naive(
    model,
    SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(),
    train_mb_size=500,
    train_epochs=1,
    eval_mb_size=100,
    evaluator=eval_plugin,
    plugins=[replay, ewc]
)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/avalanche/training/templates/base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the Naive.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


In [47]:

# Start training loop
print('Starting experiment with Replay...')

results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # Train on the current experience using EWC regularization
    res = cl_strategy_hybrid.train(experience)
    print('Training completed')

    # Evaluate on the entire test set
    print('Computing accuracy on the whole test set')
    results.append(cl_strategy_hybrid.eval(scenario.test_stream))


Starting experiment with Replay...
Start of experience:  0
Current Classes:  [4, 5]
-- >> Start of training phase << --
100%|██████████| 23/23 [00:02<00:00,  8.15it/s]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 56807.6113
	DiskUsage_MB/train_phase/train_stream/Task000 = 56807.6113
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0006
	Loss_MB/train_phase/train_stream/Task000 = 0.3774
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0087
	Time_Epoch/train_phase/train_stream/Task000 = 2.8144
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6536
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9430


/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 29.73it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 88.3124
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56807.6113
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.2725
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9562
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 28.21it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 89.2299
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 56811.6113
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.1656
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 0.7398
	Top1_Acc_Exp/eval_phase/test_stream/Task0

/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 30.24it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 88.7768
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56835.5771
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.0267
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.3084
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9296
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 27.98it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 87.2651
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 56835.5771
	Loss_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1811
	Top1_Acc_Exp/eval_phase/test_stream/Task0

...

-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 30.46it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 89.2939
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56851.5771
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.2241
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7511
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7321
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 27.49it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 88.9465
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 56851.5771
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = -0.0031
	Loss_Exp/eval_phase/test_str

/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 30.61it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 89.1147
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56875.5439
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.6467
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 1.4897
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.3095
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 31.36it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 89.0325
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 56875.5439
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.0139
	Loss_Exp/eval_phase/test_stre

/usr/local/lib/python3.10/dist-packages/avalanche/training/plugins/replay.py:123: DeprecationWarning: Call to deprecated function update (removal in version 0.7: switch to pre_adapt and post_adapt)
  self.storage_policy.update(strategy, **kwargs)


-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 19/19 [00:00<00:00, 21.67it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 82.6927
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 56891.5439
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.6537
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 1.5740
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.3026
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 20/20 [00:00<00:00, 21.47it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 79.1757
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp001 = 56891.5439
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp001 = 0.0872
	Loss_Exp/eval_phase/test_stre